In [2]:
import pandas as pd

In [3]:
df=pd.read_csv("data/final_trained_completeeee.csv")

In [4]:
#converting integer column to object
df["time_for_pickup"]=df["time_for_pickup"].astype("str")
df["Vehicle_condition"]=df["Vehicle_condition"].astype("str")
df["multiple_deliveries"]=df["multiple_deliveries"].astype("str")


In [7]:
#dropping id column
df=df.drop(labels=["id"],axis=1)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43853 entries, 0 to 43852
Data columns (total 15 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Delivery_person_Age          43853 non-null  float64
 1   time_for_pickup              43853 non-null  object 
 2   Delivery_person_Ratings      43853 non-null  float64
 3   Restaurant_latitude          43853 non-null  float64
 4   Restaurant_longitude         43853 non-null  float64
 5   Delivery_location_longitude  43853 non-null  float64
 6   Weather_conditions           43853 non-null  object 
 7   Road_traffic_density         43853 non-null  object 
 8   Vehicle_condition            43853 non-null  object 
 9   Type_of_order                43853 non-null  object 
 10  Type_of_vehicle              43853 non-null  object 
 11  multiple_deliveries          43853 non-null  object 
 12  Festival                     43853 non-null  object 
 13  City            

In [9]:
# Define which columns should be ordinal-encoded, onehot encoded and which should be scaled
numerical_cols = ['Delivery_person_Age', 'Delivery_person_Ratings',
       'Restaurant_latitude', 'Restaurant_longitude',
       'Delivery_location_longitude']
cat_ordinal_cols=["time_for_pickup","Weather_conditions","Road_traffic_density","Vehicle_condition","multiple_deliveries","Festival","City"]
cat_onehot_cols=["Type_of_order","Type_of_vehicle"]

In [10]:
# Define the custom ranking for each ordinal variable
time_pickup_cat=["5","10","15"]
weather_cat=["Sunny","Cloudy","Fog","Windy","Stormy","Sandstorms"]
traffic_condition_cat=["Low","Medium","High","Jam"]
vehicle_cat=["2","1","0"]
multiple_deliveries_cat=["0.0","1.0","2.0","3.0"]
festival_Cat=["No","Yes"]
city_cat=["Semi-Urban","Urban","Metropolitian"]
#define onehot encoded variable
vehicle_type_cat=["motorcycle","electric_scooter","scooter"]
order_type_cat=['Snack', 'Meal', 'Drinks', 'Buffet']

In [11]:
## Independent and dependent features
X=df.drop(labels=["Time_taken"],axis=1)
Y=df[["Time_taken"]]

In [12]:
#importing libraries
from sklearn.impute import SimpleImputer ## HAndling Missing Values
from sklearn.preprocessing import StandardScaler # HAndling Feature Scaling
from sklearn.preprocessing import OneHotEncoder # One hot encoding Encoding
from sklearn.preprocessing import OrdinalEncoder # Ordinal Encoding
## pipelines
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [13]:
## Numerical Pipeline
num_pipeline=Pipeline(
    steps=[
    ('imputer',SimpleImputer(strategy='median')),
    ('scaler',StandardScaler())

    ]

)



#cate ordinal
cat_ordinal_pipeline=Pipeline(
    steps=[
    ('imputer',SimpleImputer(strategy='most_frequent')),
    ("ordinalencoder",OrdinalEncoder(categories=[time_pickup_cat,weather_cat,traffic_condition_cat,vehicle_cat,multiple_deliveries_cat,festival_Cat,city_cat])),
    ("scaler",StandardScaler())
    ]

)

#Categorical onehote encoder pipeline
cat_onehot_pipeline=Pipeline(
    steps=[
    ('imputer',SimpleImputer(strategy='most_frequent')),
    ("onehotencoder",OneHotEncoder(categories=[order_type_cat,vehicle_type_cat],sparse_output=False)),
    ("scaler",StandardScaler())
    ]

)



preprocessor=ColumnTransformer([
('num_pipeline',num_pipeline,numerical_cols),
("cat_ordinal_pipeline",cat_ordinal_pipeline,cat_ordinal_cols),
("cat_onehot_pipeline",cat_onehot_pipeline,cat_onehot_cols)
])

In [14]:
## Train test split
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.30,random_state=30)

In [15]:
#fit and transform the data
X_train=pd.DataFrame(preprocessor.fit_transform(X_train))
X_test=pd.DataFrame(preprocessor.transform(X_test))
X_train.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,0.081952,-1.391154,0.024831,0.361469,0.363656,-1.220373,-0.299064,1.303501,0.003151,3.947922,7.046639,0.53658,-0.582604,-0.579745,1.748034,-0.57498,-1.184042,-0.297697,1.409184
1,0.256041,0.208743,1.257791,0.237079,0.239266,-1.220373,0.876324,-1.106277,1.227533,0.440829,-0.141912,0.53658,-0.582604,-0.579745,1.748034,-0.57498,0.844564,-0.297697,-0.709631
2,-0.788495,-0.431216,-2.242553,-3.340540,-3.340216,1.222601,-0.299064,-1.106277,1.227533,0.440829,-0.141912,0.53658,-0.582604,-0.579745,1.748034,-0.57498,0.844564,-0.297697,-0.709631
3,-0.440316,-2.031112,1.256275,0.239033,0.240275,1.222601,-0.299064,-0.303018,0.003151,0.440829,-0.141912,0.53658,-0.582604,-0.579745,-0.572071,1.73919,-1.184042,-0.297697,1.409184
4,0.081952,-0.751195,0.024527,0.362112,0.360520,0.001114,-0.299064,1.303501,0.003151,2.194376,-0.141912,0.53658,-0.582604,-0.579745,-0.572071,1.73919,0.844564,-0.297697,-0.709631


In [16]:
from sklearn.linear_model import LinearRegression,Lasso,Ridge,ElasticNet
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

In [17]:
#defining function for scores
import numpy as np
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

In [18]:
## Train multiple models

models={
    'LinearRegression':LinearRegression(),
    'Lasso':Lasso(),
    'Ridge':Ridge(),
    'Elasticnet':ElasticNet()
}
trained_model_list=[]
model_list=[]
r2_list=[]

for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(X_train,y_train)

    #Make Predictions
    y_pred=model.predict(X_test)

    mae, rmse, r2_square=evaluate_model(y_test,y_pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print('Model Training Performance')
    print("RMSE:",rmse)
    print("MAE:",mae)
    print("R2 score",r2_square*100)

    r2_list.append(r2_square)
    
    print('='*35)
    print('\n')

LinearRegression
Model Training Performance
RMSE: 6.432205858802957
MAE: 5.1319466514806935
R2 score 52.76343004947488


Lasso
Model Training Performance
RMSE: 7.007463640123568
MAE: 5.609034351642673
R2 score 43.936502247609255


Ridge
Model Training Performance
RMSE: 6.632186406042899
MAE: 5.270595358760122
R2 score 49.78055234623281


Elasticnet
Model Training Performance
RMSE: 7.115671394143951
MAE: 5.711850200185912
R2 score 42.19169290555818


